In [1]:
import pandas as pd
import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [158]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php'
browser = Browser('chrome', headless=False)

browser.visit(url)

browser.links.find_by_partial_text('All')[3].click()

In [163]:
eq_file = "../Downloads/all_month.csv"
eq_df = pd.read_csv(eq_file)
eq_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-01-04T16:19:52.500Z,35.697000,-117.527167,6.16,0.69,ml,14.0,132.0,0.08223,0.18,...,2020-01-04T16:23:43.828Z,"14km SW of Searles Valley, CA",earthquake,0.50,1.47,0.106,8.0,automatic,ci,ci
1,2020-01-04T16:11:39.447Z,59.807300,-151.430600,36.30,2.00,ml,NaN,NaN,NaN,0.83,...,2020-01-04T16:14:36.168Z,"10km NW of Fritz Creek, Alaska",earthquake,NaN,1.80,NaN,NaN,automatic,ak,ak
2,2020-01-04T15:58:11.686Z,62.899500,-148.252300,59.70,1.70,ml,NaN,NaN,NaN,0.84,...,2020-01-04T16:10:24.727Z,"65km SSE of Cantwell, Alaska",earthquake,NaN,0.40,NaN,NaN,automatic,ak,ak
3,2020-01-04T15:54:12.450Z,35.709500,-117.591333,9.70,0.96,ml,14.0,117.0,0.08913,0.12,...,2020-01-04T15:57:56.132Z,"12km NE of Ridgecrest, CA",earthquake,0.31,0.65,0.129,9.0,automatic,ci,ci
4,2020-01-04T15:37:58.184Z,60.030500,-139.105600,0.00,3.60,ml,NaN,NaN,NaN,0.88,...,2020-01-04T16:06:06.107Z,"64km NNE of Yakutat, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10470,2019-12-05T17:03:35.930Z,47.738000,-113.766167,5.72,0.38,md,6.0,208.0,0.20800,0.02,...,2019-12-06T00:39:52.590Z,"30km E of Polson, Montana",earthquake,0.59,5.55,0.204,4.0,reviewed,mb,mb
10471,2019-12-05T16:59:43.130Z,35.958833,-117.343667,3.37,0.95,ml,10.0,103.0,0.08359,0.13,...,2019-12-05T17:03:29.171Z,"22km N of Trona, CA",earthquake,0.40,0.91,0.138,9.0,automatic,ci,ci
10472,2019-12-05T16:53:55.900Z,35.740667,-117.559667,9.49,0.82,ml,22.0,114.0,0.08099,0.16,...,2019-12-05T17:13:17.778Z,"14km WSW of Searles Valley, CA",earthquake,0.28,0.64,0.131,12.0,reviewed,ci,ci
10473,2019-12-05T16:53:44.610Z,36.588333,-121.182000,3.02,2.06,md,25.0,126.0,0.01682,0.05,...,2019-12-14T09:57:04.113Z,"7km NNW of Pinnacles, CA",earthquake,0.21,0.31,0.317,16.0,reviewed,nc,nc


In [164]:
# select columns of interest
new_eq_df = eq_df[['time','latitude','longitude','depth','mag','magType','place']].copy()

# create new column column for state
new_eq_df['state'] = new_eq_df['place'].str.split(',').str[1].str.strip()

# set state to index so that the loc function can be used to find all earthquakes that happened in oklahoma
new_eq_df = new_eq_df.set_index('state')

# execute loc function
new_eq_df = new_eq_df.loc['Oklahoma']

# reset the index so that state will be pushed into the dataframe
new_eq_df = new_eq_df.reset_index()

In [165]:
# make connection to database and connect the client
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# create database
db = client.earthquakesDB

# insert rows of the dataframe as a dictionary to the database with column names as the key
db.earthquakes.insert_many(new_eq_df.to_dict('records'))

# Use Browser to download all earthquakes that happened that day

In [151]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php'
browser = Browser('chrome', headless=False)

browser.visit(url)

In [2]:
browser.links.find_by_partial_text('All')[1].click()

# Read csv file into dataframe

In [6]:
eq_file = "../Downloads/all_day.csv"
eq_df = pd.read_csv(eq_file)
eq_df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-01-03T01:49:15.980Z,38.154800,-118.283200,10.50,1.90,ml,14.0,121.28,0.14900,0.04,...,2020-01-03T01:53:37.693Z,"50km SE of Hawthorne, Nevada",earthquake,NaN,2.70,NaN,NaN,automatic,nn,nn
1,2020-01-03T01:19:10.530Z,40.247700,139.329800,193.71,4.90,mb,NaN,121.00,2.35700,0.58,...,2020-01-03T01:36:16.040Z,"59km W of Noshiro, Japan",earthquake,4.50,7.40,0.043,170.0,reviewed,us,us
2,2020-01-03T01:10:44.300Z,59.725100,-136.856900,0.00,2.20,ml,NaN,NaN,NaN,0.89,...,2020-01-03T01:18:53.112Z,"92km WNW of Skagway, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
3,2020-01-03T01:10:34.376Z,61.467900,-149.975200,45.20,1.30,ml,NaN,NaN,NaN,0.47,...,2020-01-03T01:13:11.791Z,"6km S of Big Lake, Alaska",earthquake,NaN,0.90,NaN,NaN,automatic,ak,ak
4,2020-01-03T01:01:29.000Z,36.137501,-117.866165,2.49,2.38,ml,27.0,79.00,0.02189,0.14,...,2020-01-03T01:05:19.085Z,"13km NE of Coso Junction, CA",earthquake,0.27,0.41,0.256,27.0,automatic,ci,ci


# Data cleaning

In [14]:
# select columns of interest
new_eq_df = eq_df[['time','latitude','longitude','depth','mag','magType','place']].copy()

# create new column column for state
new_eq_df['state'] = new_eq_df['place'].str.split(',').str[1].str.strip()

# set state to index so that the loc function can be used to find all earthquakes that happened in oklahoma
new_eq_df = new_eq_df.set_index('state')

# execute loc function
new_eq_df = new_eq_df.loc['Oklahoma']

# reset the index so that state will be pushed into the dataframe
new_eq_df = new_eq_df.reset_index()

# Read Salt Water Injection Well Data into Data Frame

In [143]:
well_file = "Resources/InjectionWells.csv"
# API# was found to be a float and would show up in the data fram in scientific notation. This is improper formatting.
# String is a proper dtype for API# because API#s are unique identifier of oil wells, therefore, no mathematical operations
# would be performed on numbers
dtypes = {'API#': 'str'}
well_df = pd.read_csv(well_file, dtype=dtypes)
well_df.head()

,API#,Operator,Operator ID,WellType,WellName,WellNumber,OrderNumbers,Approval Date,County,Sec,...,Rng,QQQQ,LAT,LONG,PSI,BBLS,ZONE,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,3500300026,PHOENIX PETROCORP INC,19499.0,2R,SE EUREKA UNIT-TUCKER #1,21,133856.0,9/6/1977,ALFALFA,13,...,10W,C-SE SE,36.900324,-98.21826,"2,500",300,CHEROKEE,NaN,NaN,NaN
1,3500300163,CHAMPLIN EXPLORATION INC,4030.0,2R,CHRISTENSEN,1,470258.0,11/27/2002,ALFALFA,21,...,09W,C-NW NW,36.896636,-98.17772,"2,400","1,000",RED FORK,NaN,NaN,NaN
2,3500320786,LINN OPERATING INC,22182.0,2R,NE CHEROKEE UNIT,85,329426.0,8/19/1988,ALFALFA,24,...,11W,SE NE,36.806113,-98.32584,"1,050","1,000",RED FORK,NaN,NaN,NaN
3,3500321074,SANDRIDGE EXPLORATION & PRODUCTION LLC,22281.0,2R,VELMA,2-19,281652.0,7/11/1985,ALFALFA,19,...,10W,SW NE NE SW,36.888589,-98.31853,"3,152","1,000",RED FORK,NaN,NaN,NaN
4,3500321107,CHAMPLIN EXPLORATION INC,4030.0,2R,GRAY,1A,470257.0,11/27/2002,ALFALFA,20,...,09W,SE SW NW,36.892128,-98.19462,"1,000","2,400",RED FORK,NaN,NaN,NaN


In [144]:
# change api# column name to APInumber
well_df = well_df.rename(columns={'API#':'APInumber'})

In [145]:
# copy columns of interest
wells = well_df[['APInumber','WellType','WellName','County','LAT','LONG']].copy()

In [146]:
wells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11126 entries, 0 to 11125
Data columns (total 6 columns):
APInumber    11125 non-null object
WellType     11125 non-null object
WellName     11124 non-null object
County       11125 non-null object
LAT          11125 non-null float64
LONG         11125 non-null float64
dtypes: float64(2), object(4)
memory usage: 521.7+ KB


In [147]:
# seems like two wells are missing information
wells = wells.dropna()

# Push Data to MongoDB

In [22]:
# make connection to database and connect the client
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [23]:
# create database
db = client.earthquakesDB

In [37]:
# insert rows of the dataframe as a dictionary to the database with column names as the key
db.earthquakes.insert_many(new_eq_df.to_dict('records'))

In [150]:
# insert rows of the dataframe as a dictionary to the database with column names as the key
db.inj_wells.insert_many(wells.to_dict('records'))